# Méthode de bissection de Givens

In [1]:
import numpy as np
import numpy.random as rd
import matplotlib.pyplot as plt
import sys

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Rappels

Attention, les notations sont légèrement différentes de celles du cours. 

* On considère une matrice tridiagonale symétrique réelle

$$
  A =
  \begin{pmatrix}
    a_1 & b_1 & 0 & \ldots & 0\\
    b_1 & a_2 & b_2 & \ddots & \vdots \\
    0 & b_2 & \ddots & \ddots & 0\\
    \vdots & \ddots & b_{n-2} & a_{n-1} & b_{n-1} \\
    0 & \ldots & 0 & b_{n-1} & a_n
  \end{pmatrix},
$$

 et on suppose, sans perdre de généralité, que $b_i\neq0$ pour $1\leq i\leq n{-}1$.

* Etant donnée une valeur $\lambda\in\mathbb{R}$, on construit un $(n+1)-$uplet $(p_i(\lambda))_{0\leq i\leq n}$ par récurrence :

$$
\left\lbrace
\begin{aligned}
p_0(\lambda) & = 1,\\
p_1(\lambda) & = a_1 - \lambda,\\
p_i(\lambda) & = (a_i-\lambda) p_{i-1}(\lambda) - b_{i-1}^2 p_{i-2}(\lambda), && 2\leq i\leq n.
\end{aligned}
\right.
$$ 

Un point essentiel est que deux éléments consécutifs ne peuvent pas s'annuler simultanément. 

* Grace à la propriété précédente, on peut alors définir pour $0 \leq i \leq n$

$$
s_i(\lambda) = \left\lbrace
\begin{aligned}
&\text{signe de }p_i(\lambda) &\text{ si }p_i(\lambda)\neq0,\\
&\text{signe de }p_{i-1}(\lambda) &\text{ si }p_i(\lambda)=0.
\end{aligned} \right.
$$

Ainsi chaque $s_i(\lambda)$ est soit égal à $+1$ soit égal à $-1$ : il ne peut pas être égal à $0$.

* On construit finalement $V(\lambda)$ le nombre de changements de signe entre deux éléments consécutifs de 

$$
S(\lambda) = (s_0(\lambda), s_1(\lambda), \dots, s_n(\lambda))
.$$

_Par exemple, si $S(\lambda)=\{1,1,-1,1,1\}$, on a $V(\lambda)=2$._

 Le résultat crucial est que $V(\lambda)$ est égal au nombre de valeurs propres de la matrice $A$ qui sont strictement inférieures à $\lambda$.

Dans cet exercice, nous allons tester cette méthode sur la matrice du Laplacien de taille $n$ définie par

$$
A = (n+1)^2 \begin{pmatrix}
2&-1&0&\ldots&0\\
-1&2&-1&\ddots&\vdots\\
0&\ddots&\ddots&\ddots&0\\
\vdots&\ddots&-1&2&-1\\
0&\ldots&0&-1&2
\end{pmatrix}.
$$

### Question 1

> Proposez une fonction `G_test` qui prend en argument une matrice $A$ et qui teste si elle est de la bonne forme, c'est-à-dire symétrique, tridiagonale et avec des éléments non diagonaux différents de 0. On pourra penser à utiliser les fonctions `np.triu` et `np.tril`.

> Testez la fonction `G_test` sur la matrice du Laplacien de taille $n=4$.

In [2]:
def G_test(A):
    if(np.diag(A,-1).any() != np.diag(A,1).any()):
        return False
    
    for i in range (np.shape(A)[0] -1 ):
        if(np.diag(A,-1)[i] == 0):
            return False
        if(np.diag(A,1)[i] == 0):
            return False
        for j in range(np.shape(A)[0] -1 ):
            if((A - np.tril(A,1))[i,j] !=0):
                return False
            if((A-np.triu(A,-1))[i,j] != 0):
                return False
            
        
    return True     
   
    

In [3]:
def Laplacien(n):
    A = 2*np.eye(n,n) - np.eye(n,n,-1) - np.eye(n,n,1)
    A*= (n+1)**2
    
    return A

A = np.ones([8,8])

print(A - np.tril(A,1))

[[0. 0. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


In [4]:
G_test(Laplacien(80))

True

### Question 2

> Proposez une fonction `G_p` qui :
* prend en argument une matrice $A$ et un réel $\lambda$
* retourne $(p_0(\lambda), \ldots, p_n(\lambda))$ sous la forme d'un vecteur `np.array` de taille $n+1$.

> Testez votre fonction avec la matrice du Laplacien de taille $n=4$. Vous devriez obtenir les valeurs suivantes pour $\lambda = 50$ :

> ```[ 1.00000e+00  0.00000e+00 -6.25000e+02 -0.00000e+00  3.90625e+05]```


In [46]:
def G_p(A,λ):
    p = []
    p.append(1)
    p.append(A[0,0] - λ)
    for i in range (2,np.shape(A)[0]+1  ):
        p.append((A[i-1,i-1] - λ)*p[i-1] - p[i-2] * A[i-1,i-2]**2)
    
    p = np.array(p)
        
    return p

In [47]:
G_p(Laplacien(4),50)

array([ 1.00000e+00,  0.00000e+00, -6.25000e+02, -0.00000e+00,
        3.90625e+05])

### Question 3

> Proposez une fonction `G_s` qui prend en argument le vecteur $p = (p_0(\lambda),\ldots,p_n(\lambda))$ et qui retourne le vecteur $s = (s_0(\lambda),\ldots,s_n(\lambda))$ associé.

> Vérifiez que la fonction est opérationnelle sur le vecteur $p$ obtenu à la question précédente.

In [48]:
def G_s(p):
    s=[]
    for i in range (np.shape(p)[0]):
        if(p[i] > 0):
            s.append(1)
        if(p[i]< 0):
            s.append(-1)
        if(p[i] ==0):
            s.append(s[i-1])
    return s

In [49]:
G_s(G_p(Laplacien(4),50))

[1, 1, -1, -1, 1]

### Question 4

> Proposez une fonction `G_V` qui prend en argument le vecteur $s$ et qui retourne le nombre de changement de signe $V(\lambda)$.

> Vérifiez que la fonction est opérationnelle sur le vecteur $s$ obtenu à la question précédente.

In [54]:
def G_V(s):
    v=0
    for i in range (1,np.shape(s)[0]):
        v +=(np.abs(s[i]-s[i-1]))
    v *= 0.5
    return v

In [55]:
G_V(G_s(G_p(Laplacien(4),50)))

2.0

Nous allons à présent utiliser ces fonctions afin de déterminer un encadrement (théoriquement aussi précis que souhaité) de n'importe quelle valeur propre de $A$ en utilisant un algorithme de dichotomie. Nous choisissons donc un entier $p$ compris entre $1$ et $n$ et nous cherchons à déterminer la $p-$ième valeur propre de la matrice du Laplacien de taille $n$.

En premier lieu, il est nécessaire de déterminer un premier encadrement grossier du spectre de la matrice, disons 

$$
\operatorname{Spec}(A) \subset \{z\in\mathbb{C}\,:\; |z| \leq \Vert A \Vert_2\}\,.
$$

Nous construisons alors trois suites de la façon suivante :

$$
\alpha_0 = -\Vert A \Vert_2  \quad  \text{et}  \quad  \beta_0 = \Vert A \Vert_2
.$$

Etant donnés $\alpha_i$ et $\beta_i$, nous calculons $\gamma_i = (\alpha_i + \beta_i)/2$ et $V(\gamma_i)$. 
Puis nous définissons $\alpha_{i+1}$ et $\beta_{i+1}$ ainsi :

$$
\alpha_{i+1} = \left\lbrace
\begin{aligned}
&\alpha_i&&\text{si } V(\gamma_i)\geq p,\\
&\gamma_i&&\text{sinon},
\end{aligned}
\right.
\qquad
\beta_{i+1} = \left\lbrace
\begin{aligned}
&\gamma_i&&\text{si } V(\gamma_i)\geq p,\\
&\beta_i&&\text{sinon}.
\end{aligned}
\right.
$$

A chaque étape, on construit un nouvel intervalle contenant la $p-$ième valeur propre et dont la longueur est divisée par 2. Nous stoppons l'algorithme lorsque cette longueur a atteint une valeur cible assez petite ($10^{-10}$ par exemple).

### Question 5

> Proposez une fonction `Givens` qui
* prend en argument une matrice $A$ de taille $n$, un entier $p$ compris entre $1$ et $n$ et un réel $\varepsilon>0$,
* vérifie que la matrice $A$ est de la bonne forme pour pouvoir appliquer l'algorithme précédemment décrit,
* et retourne un encadrement de la $p-$ième valeur propre de $A$ de longueur inférieur (ou égal) à $\varepsilon$.


In [112]:
def Givens(A,p,ε):
    if(G_test(A)==False):
        return "Pas bonne forme"
    
    a = np.round((-np.linalg.norm(A)),11)
    b = np.round((np.linalg.norm(A)),11)
    g = 0.5*(a+b)
    v = G_V(G_s(G_p(A,g)))
    
    i= 0
    
    while(np.abs(a-b) > ε):
        if(v<p):
            a = g
        if(v >= p):
            b = g
        g = 0.5*(a+b)
        v = G_V(G_s(G_p(A,g)))
        i+=1
    
    a = np.round(a,101)
    b= np.round(b,101)
    return a,b
    
    

### Question 6

> Testez la fonction `Givens` en déterminant un intervalle de longueur inférieur à $10^{-10}$ contenant chacune des valeurs propres de la matrice du Laplacien de taille $n=10$.
> Vérifiez que les valeurs propres exactes sont bien dans chacun des intervalles trouvés.

_Nous rappelons que les valeurs propres exactes de la matrice du Laplacien de taille $n$ s'écrivent_

$$ \lambda_i = 4(n+1)^2 \sin^2\left(\frac{i\pi}{2(n+1)}\right), \qquad 1\leq i\leq n.$$

In [ ]:
eps = 1e-1
n= 10
ld = []
G = []
for i in range(1,n+1):
    ld.append(4*pow((n+1),2) * pow(np.sin(i * np.pi*0.5/(n+1)),2))
    G.append(Givens(Laplacien(n),i,eps))


In [ ]:
print(ld)
print(" \n")
print(G)


In [69]:
np.linalg.eig(Laplacien(n))

(array([474.19729961, 445.58335495, 400.47629761, 342.53043315,
        276.44019086, 207.55980914,   9.80270039,  38.41664505,
         83.52370239, 141.46956685]),
 array([[ 0.12013117,  0.23053002,  0.3222527 ,  0.38786839, -0.42206128,
          0.42206128, -0.12013117,  0.23053002, -0.3222527 , -0.38786839],
        [-0.23053002, -0.38786839, -0.42206128, -0.3222527 ,  0.12013117,
          0.12013117, -0.23053002,  0.38786839, -0.42206128, -0.3222527 ],
        [ 0.3222527 ,  0.42206128,  0.23053002, -0.12013117,  0.38786839,
         -0.38786839, -0.3222527 ,  0.42206128, -0.23053002,  0.12013117],
        [-0.38786839, -0.3222527 ,  0.12013117,  0.42206128, -0.23053002,
         -0.23053002, -0.38786839,  0.3222527 ,  0.12013117,  0.42206128],
        [ 0.42206128,  0.12013117, -0.38786839, -0.23053002, -0.3222527 ,
          0.3222527 , -0.42206128,  0.12013117,  0.38786839,  0.23053002],
        [-0.42206128,  0.12013117,  0.38786839, -0.23053002,  0.3222527 ,
          0.322